In [1]:
%run 01_setup.ipynb

23/11/20 13:57:11 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [22]:
import time
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, max, min, col, row_number

In [3]:
hw_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- geoHash: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- wthr_date: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [36]:
hw_clear = hw_df.groupBy('id', col('address').alias('hotel_name'), 'city', 'country', 'year', 'month') \
    .agg(
        max('avg_tmpr_c').alias('max_t'),
        min('avg_tmpr_c').alias('min_t'),
        (max('avg_tmpr_c') - min('avg_tmpr_c')).alias('tmpr_diff')
    ).where('tmpr_diff > 0')

In [37]:
start_time = time.time()

window = Window.partitionBy('year', 'month').orderBy(col('tmpr_diff').desc())
result = hw_clear.withColumn("rank", row_number().over(window)).where(col("rank").cast("int") < 11)
result.show()

print("Execution time is ", round(time.time() - start_time, 2), "sec.")

+-------------+--------------------+--------------+-------+----+-----+-----+-----+------------------+----+
|           id|          hotel_name|          city|country|year|month|max_t|min_t|         tmpr_diff|rank|
+-------------+--------------------+--------------+-------+----+-----+-----+-----+------------------+----+
|1571958030336|Little Belt Inn N...|       Neihart|     US|2016|   10| 10.8| -7.7|              18.5|   1|
|  77309411328|           The Point|  Saranac Lake|     US|2016|   10| 19.2|  1.0|              18.2|   2|
| 695784701957|         Rodeway Inn|      Gillette|     US|2016|   10| 15.2| -1.1|              16.3|   3|
| 369367187456|    Greektown Casino|       Detroit|     US|2016|   10| 21.1|  4.9|16.200000000000003|   4|
|  77309411335|Clarion Inn Stras...|     Strasburg|     US|2016|   10| 20.7|  5.8|14.899999999999999|   5|
|1520418422787|  Brooks Donald L Jr|     Strasburg|     US|2016|   10| 20.7|  5.8|14.899999999999999|   6|
|1013612281860|     Days Inn Geneva| 

In [39]:
result.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (16)
+- Filter (15)
   +- Window (14)
      +- WindowGroupLimit (13)
         +- Sort (12)
            +- Exchange (11)
               +- WindowGroupLimit (10)
                  +- Sort (9)
                     +- Filter (8)
                        +- HashAggregate (7)
                           +- Exchange (6)
                              +- HashAggregate (5)
                                 +- InMemoryTableScan (1)
                                       +- InMemoryRelation (2)
                                             +- * ColumnarToRow (4)
                                                +- Scan parquet  (3)


(1) InMemoryTableScan
Output [7]: [address#0, avg_tmpr_c#1, city#3, country#4, id#6, year#11, month#12]
Arguments: [address#0, avg_tmpr_c#1, city#3, country#4, id#6, year#11, month#12]

(2) InMemoryRelation
Arguments: [address#0, avg_tmpr_c#1, avg_tmpr_f#2, city#3, country#4, geoHash#5, id#6, latitude#7, longitude#8, name#9, wthr_date#1